### 1. Blocking code

In [ ]:
import time

def solve_problem(problem: str) -> str:
    if problem == "easy":
        time.sleep(1)
        return "easy"
    if problem == "hard":
        time.sleep(2)
        return "hard"
    raise ValueError(f"Too hard: {problem}")

def solve_all() -> None:
    start = time.time()
    solution_1 = solve_problem("easy")
    solution_2 = solve_problem("hard")
    solved = [solution_1, solution_2]
    end = time.time()
    print(f"Solved problems: {solved}, time: {end - start:.1f}")

In [ ]:
solve_all()

### 2. Simple concurrent code

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def solve_all_thread_pool() -> None:
    start = time.time()
    with ThreadPoolExecutor(max_workers=2) as executor:
        solution_1 = executor.submit(solve_problem, "easy")
        solution_2 = executor.submit(solve_problem, "hard")
        solved = [solution_1.result(), solution_2.result()]
    end = time.time()
    print(f"Solved problems: {solved}, time: {end - start:.1f}")

In [ ]:
solve_all_thread_pool()


### 3. Using wait

In [ ]:
from concurrent.futures import wait, ALL_COMPLETED


def solve_all_wait() -> None:
    start = time.time()
    with ThreadPoolExecutor(max_workers=2) as executor:
        done, not_done = wait(
            [
                executor.submit(solve_problem, "easy"),
                executor.submit(solve_problem, "hard"),
            ],
            return_when=ALL_COMPLETED,
        )
        solved = [future.result() for future in done]
    end = time.time()
    print(f"Solved problems: {solved}, time: {end - start:.1f}")

In [ ]:
solve_all_wait()

### 4. Proces results individually as soon as they are available

In [ ]:
from concurrent.futures import FIRST_COMPLETED


def solve_problem_method(problem: str, method: str = "naive") -> dict[str, str]:
    match (problem, method):
        case "easy", "naive":
            time.sleep(1)
        case "easy", "complex":
            time.sleep(5)
        case "hard", "naive":
            time.sleep(3)
            raise ValueError(f"Too hard: {problem =}, {method =}")
        case "hard", "complex":
            time.sleep(7)
        case _:
            raise ValueError(f"Cannot solve: {problem =}, {method =}")

    return {"problem": problem, "method": method, "solution": f"{problem} ({method})"}

def solve_all_method_wait() -> None:
    start = time.time()
    with ThreadPoolExecutor(max_workers=4) as executor:
        tasks = {
            problem: {
                method: executor.submit(solve_problem_method, problem, method)
                for method in ["naive", "complex"]
            }
            for problem in ["easy", "hard"]
        }
        not_done = [task for problem_tasks in tasks.values() for task in problem_tasks.values()]
        while not_done:
            done, not_done = wait(not_done, return_when=FIRST_COMPLETED)
            for future in done:
                end = time.time()
                try:
                    result = future.result()
                except Exception as e:
                    end = time.time()
                    print(f"Error in solustion: {e}, time: {end - start:.1f}")
                    continue

                print(f"Solved: {result['solution']}, time: {end - start:.1f}")
                # try to cancel any pending tasks for the same problem
                solved_problem = result["problem"]
                for method, task_to_cancel in tasks[solved_problem].items():
                    if task_to_cancel in not_done:
                        cancelled = task_to_cancel.cancel()
                        if cancelled:
                            print(f"Cancelled problem: {solved_problem}, method: {method}")
                        else:
                            print(f"Cannot cancel problem: {solved_problem}, method: {method}")

In [ ]:

solve_all_method_wait()

## Asyncio

### 1. Simple concurrent code with asyncio


In [ ]:
import asyncio
import time

async def solve_problem_async(problem: str) -> str:
    if problem == "easy":
        await asyncio.sleep(1)
        return "easy"
    if problem == "hard":
        await asyncio.sleep(2)
        return "hard"
    raise ValueError(f"Too hard: {problem}")

async def solve_all_async() -> None:
    start = time.time()
    solution_1 = await solve_problem_async("easy")
    solution_2 = await solve_problem_async("hard")
    solved = [solution_1, solution_2]
    end = time.time()
    print(f"Solved problems: {solved}, time: {end - start:.1f}")


In [ ]:
await solve_all_async()

### 2. Asyncio with asyncio.gather

In [ ]:
async def solve_all_async_gather() -> None:
    start = time.time()
    solved = await asyncio.gather(
        solve_problem_async("easy"),
        solve_problem_async("hard")
    )
    end = time.time()
    print(f"Solved problems: {solved}, time: {end - start:.1f}")

In [ ]:

await solve_all_async_gather()